In [11]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from collections import defaultdict
import math

In [ ]:
# Загружаем данные из CSV файла
csv_path = "train.csv"
df = pd.read_csv(csv_path)
df.head(5)

,id,phrase,split_phrase,ipa,split_ipa
0,0,取り残され,取り残さ れ,t o ɾʲ i n o k o s a ɾ e,t o ɾʲ i n o k o s a <space> ɾ e
1,1,ブスかわ,ブス か わ,b ɯ s ɨ k a w a,b ɯ s ɨ <space> k a <space> w a
2,2,早鐘を打つ,早鐘 を 打つ,h a j a ɡ a n e o ː t s ɨ,h a j a ɡ a n e <space> o <space> ː t s ɨ
3,3,マニラ紙,マニラ 紙,m a ɲ i ɾ a ɕ i,m a ɲ i ɾ a <space> ɕ i
4,4,すいこめば,すいこめ ば,s ɨ i k o m e b a,s ɨ i k o m e <space> b a


In [13]:
# Убираем все пробелы из столбцов ipa и split_ipa
for col in ["ipa", "split_ipa"]:
    df[col] = df[col].str.replace(" ", "", regex=False)

print("Пример после очистки пробелов:")
df.head(5)

Пример после очистки пробелов:


,id,phrase,split_phrase,ipa,split_ipa
0,0,取り残され,取り残さ れ,toɾʲinokosaɾe,toɾʲinokosa<space>ɾe
1,1,ブスかわ,ブス か わ,bɯsɨkawa,bɯsɨ<space>ka<space>wa
2,2,早鐘を打つ,早鐘 を 打つ,hajaɡaneoːtsɨ,hajaɡane<space>o<space>ːtsɨ
3,3,マニラ紙,マニラ 紙,maɲiɾaɕi,maɲiɾa<space>ɕi
4,4,すいこめば,すいこめ ば,sɨikomeba,sɨikome<space>ba


In [14]:
# Общая информация
print("Размер датафрейма:", df.shape)
print("\nИнформация:")
print(df.info())

Размер датафрейма: (5874, 5)

Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            5874 non-null   int64 
 1   phrase        5874 non-null   object
 2   split_phrase  5874 non-null   object
 3   ipa           5874 non-null   object
 4   split_ipa     5874 non-null   object
dtypes: int64(1), object(4)
memory usage: 229.6+ KB
None


In [15]:
# Длины строк в фонемах 
ipa_lengths = df["ipa"].apply(len)

print("Распределение длины фонем (ipa):")
print(ipa_lengths.describe())

Распределение длины фонем (ipa):
count    5874.000000
mean       10.615254
std         3.056884
min         4.000000
25%         8.000000
50%        10.000000
75%        12.000000
max        32.000000
Name: ipa, dtype: float64


In [16]:
# Количество слов в японской записи предложения (split_phrase)
word_counts = df["split_phrase"].str.split().apply(len)
print("Распределение количества слов (split_phrase):")
print(word_counts.describe())

Распределение количества слов (split_phrase):
count    5874.000000
mean        2.296902
std         0.606200
min         2.000000
25%         2.000000
50%         2.000000
75%         2.000000
max         7.000000
Name: split_phrase, dtype: float64


In [17]:
# Длины отдельных слов по японским символам (иероглифам) в split_phrase
all_words = df["split_phrase"].str.split()
word_lengths = all_words.explode().str.len()
print("Распределение длины слова (в иероглифах):")
print(word_lengths.describe())

Распределение длины слова (в иероглифах):
count    13492.000000
mean         2.068485
std          1.043233
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          9.000000
Name: split_phrase, dtype: float64


In [18]:
# Длины отдельных слов по символам в split_ipa
all_words = df["split_ipa"].str.split("<space>")
word_lengths = all_words.explode().str.len()
print("Распределение длины фонем (в символах):")
print(word_lengths.describe())

Распределение длины фонем (в символах):
count    13492.000000
mean         4.621554
std          2.284612
min          1.000000
25%          3.000000
50%          4.000000
75%          6.000000
max         18.000000
Name: split_ipa, dtype: float64


In [19]:
# Для всех данных вычислим среднее количество фонем на один японский символ
total_ipa_length = df["ipa"].str.len().sum()
total_char_count = df["phrase"].str.len().sum()
avg_phonemes_per_char = total_ipa_length / total_char_count

print(f"Среднее количество фонем на один японский символ: {avg_phonemes_per_char:.3f}")

Среднее количество фонем на один японский символ: 2.234


In [ ]:
# Подготовка данных для обучения модели

# Создаем функцию, которая разбивает фразу и извлекает отдельные японские символы (иероглифы)
def split_phrase_to_symbols(split_phrase: str):
    words = split_phrase.split()
    symbols = []
    word_ids = []
    for word_idx, word in enumerate(words):
        chars = list(word)
        symbols.extend(chars)
        word_ids.extend([word_idx] * len(chars))
    return words, symbols, word_ids # Возвращает список слов, список символов и список индексов слов для каждого иероглифа


# Подготовка данных
samples = []    # структура по каждому предложению (уникальные символы, макс. длина фонем и т.д.)
symbol_vocab = set()    # множество всех уникальных символов 
max_ratio = 0.0     # максимальное отношение длины фонем к числу символов (от него зависит ограничение на длину сегмента)
max_ipa_len = 0     # максимальная длина фонем
max_symbol_len = 0   # максимальное количество символов в split_phrase

# Обработка каждого предложения из датафрейма
for row in df.itertuples(index=False):
    words, symbols, word_ids = split_phrase_to_symbols(row.split_phrase)    
    if not symbols or not isinstance(row.ipa, str) or len(row.ipa) == 0:    
        continue

    # Вычисляем отношение длины фонем к количеству символов для определения максимальной длины
    ipa_chars = list(row.ipa)  
    ratio = len(ipa_chars) / len(symbols)   # сколько фонем приходится на один японский символ
    max_ratio = max(max_ratio, ratio)  
    max_ipa_len = max(max_ipa_len, len(ipa_chars))  
    max_symbol_len = max(max_symbol_len, len(symbols))  

    # Сохраняем данные для каждого предложения
    samples.append(
        {
            "id": int(row.id),
            "split_phrase": row.split_phrase,
            "words": words,
            "symbols": symbols,
            "word_ids": word_ids,  # индекс слова для каждого символа
            "ipa_chars": ipa_chars,  # фонемная строка, разбитая посимвольно
            "ipa_str": row.ipa,
            "split_ipa_true": row.split_ipa,  # разметка только для оценки
        }
    )
    symbol_vocab.update(symbols)


max_chunk_len = max(int(np.ceil(max_ratio)) + 1, 4)

print(f"Всего предложений: {len(samples)}")
print(f"Всего уникальных символов: {len(symbol_vocab)}")
print(f"Максимальная длина фонем: {max_ipa_len}, максимальное количество символов в split_phrase: {max_symbol_len}")
print(f"Максимальное отношение длины фонем к числу символов: {max_ratio:.2f}")

print(f"\nБудем ограничивать длину одного символа значением {max_chunk_len}")

Всего предложений: 5874
Всего уникальных символов: 1811
Максимальная длина фонем: 32, максимальное количество символов в split_phrase: 13
Максимальное отношение длины фонем к числу символов: 4.67

Будем ограничивать длину одного символа значением 6


In [ ]:
# Реализация скрытой полу-марковской модели (HSMM) для сегментации фонем
# Модель обучает распределение длины сегментов фонем для каждого японского символа (иероглифа)

# Функция для суммирования вероятностей в логарифмическом пространстве
def logsumexp_pair(a, b):
    if not math.isfinite(a):
        return b
    if not math.isfinite(b):
        return a
    if a > b:
        return a + math.log1p(math.exp(b - a))
    return b + math.log1p(math.exp(a - b))

In [ ]:
# Функция для инициализации длин сегментов фонем для каждого символа
def greedy_length_initialization(num_symbols: int, ipa_len: int, max_len: int):
    # num_symbols - количество японских символов
    # ipa_len - общая длина фонемной последовательности
    # max_len - максимальная длина сегмента фонем для одного символа
    lengths = []  
    remaining = ipa_len 
    for idx in range(num_symbols):
        symbols_left = num_symbols - idx
        if symbols_left == 0:
            break

        # Вычисляем минимальную и максимальную допустимую длину для текущего символа
        min_len = max(1, remaining - (symbols_left - 1) * max_len) 
        max_len_allowed = min(max_len, remaining - (symbols_left - 1)) 
        if max_len_allowed < min_len:
            max_len_allowed = min_len

        # Берем среднее значение с округлением
        avg = remaining / symbols_left
        length = int(round(avg))
        length = max(min_len, min(max_len_allowed, length)) 
        lengths.append(length)
        remaining -= length

    # Распределяем остаток обратно по символам, если он есть
    if remaining != 0:
        for idx in range(len(lengths) - 1, -1, -1):
            take = min(max_len - lengths[idx], remaining)
            lengths[idx] += take
            remaining -= take
            if remaining == 0:
                break
    if remaining != 0:
        raise ValueError("Не удалось корректно разложить длину IPA по иероглифам")
    return lengths

In [ ]:
# Скрытая полу-марковская модель для сегментации фонем
class LengthHSMM:
    # Инициализация модели
    def __init__(self, symbol_vocab, max_len=6, length_prior=0.1):
        # symbol_vocab - множество всех уникальных японских символов
        # max_len - максимальная длина сегмента для одного символа
        # length_prior - априорная вероятность для сглаживания распределений
        self.symbol_vocab = symbol_vocab
        self.max_len = max_len
        self.length_prior = length_prior
        self.length_log_probs = {}  # Логарифмы вероятностей длин для каждого символа
        self.initialized = False # Флаг, указывающий, инициализирована ли модель (для контроля состояния модели)

    # Инициализация параметров модели на основе жадной сегментации - подсчитывает частоты длин сегментов для каждого японского символа
    def initialize_params(self, samples):
        counts = defaultdict(lambda: np.full(self.max_len, self.length_prior)) 
        for sample in samples:
            symbols = sample["symbols"]
            ipa_len = len(sample["ipa_chars"])
            if not symbols:
                continue
            # Получаем начальную сегментацию жадным алгоритмом
            lengths = greedy_length_initialization(len(symbols), ipa_len, self.max_len)
            # Подсчитываем частоты длин фонем для каждого символа
            for ch, length in zip(symbols, lengths):
                if 1 <= length <= self.max_len:
                    counts[ch][length - 1] += 1.0
        # Нормализуем частоты в вероятности и сохраняем в логарифмическом виде
        for ch in self.symbol_vocab:
            if ch not in counts:
                counts[ch] = np.full(self.max_len, self.length_prior)
            probs = counts[ch] / counts[ch].sum()
            self.length_log_probs[ch] = np.log(probs)
        self.initialized = True

    # Получение логарифмических вероятностей длин сегментов фонем для заданного японского символа
    def _get_log_length_probs(self, char):
        if char not in self.length_log_probs: 
            self.symbol_vocab.add(char)
            uniform = np.full(self.max_len, 1.0 / self.max_len)
            self.length_log_probs[char] = np.log(uniform)
        return self.length_log_probs[char]

    # Прямой проход алгоритма для вычисления вероятностей
    def _forward_pass(self, symbols, obs_len):
        # obs_len - длина фонемной последовательности
        T = len(symbols)    
        alpha = np.full((T + 1, obs_len + 1), -np.inf) 
        alpha[0, 0] = 0.0   
        for t in range(T):
            char = symbols[t]
            log_len = self._get_log_length_probs(char)
            # Обработка каждой позиции в фонемах
            for pos in range(obs_len + 1):
                current = alpha[t, pos]
                if not math.isfinite(current):
                    continue
                # Перебираем возможные длины сегмента для текущего символа
                max_d = min(self.max_len, obs_len - pos) 
                for d in range(1, max_d + 1): 
                    log_prob = log_len[d - 1]
                    if not math.isfinite(log_prob):
                        continue
                    next_pos = pos + d
                    # Обновляем вероятность для численной устойчивости
                    alpha[t + 1, next_pos] = logsumexp_pair(
                        alpha[t + 1, next_pos], current + log_prob
                    )
        return alpha

    # Обратный проход алгоритма для вычисления вероятностей
    def _backward_pass(self, symbols, obs_len):
        T = len(symbols)
        beta = np.full((T + 1, obs_len + 1), -np.inf)
        beta[T, obs_len] = 0.0  
        for t in range(T - 1, -1, -1):
            char = symbols[t] 
            log_len = self._get_log_length_probs(char)
            for pos in range(obs_len, -1, -1):  # Идем от конца к началу
                max_d = min(self.max_len, obs_len - pos)
                if max_d == 0:  
                    continue
                value = -np.inf
                # Перебираем возможные длины сегмента
                for d in range(1, max_d + 1):
                    next_pos = pos + d
                    log_prob = log_len[d - 1]
                    if not math.isfinite(log_prob) or not math.isfinite(
                        beta[t + 1, next_pos]
                    ):
                        continue
                    value = logsumexp_pair(value, log_prob + beta[t + 1, next_pos])
                beta[t, pos] = value
        return beta

    # E-шаг алгоритма EM (Expectation и Maximization) - вычисление ожидаемых частот длин сегментов для каждого символа на основе текущих параметров модели   
    def expectation_step(self, samples):
        length_counts = defaultdict(lambda: np.zeros(self.max_len))
        total_loglike = 0.0 
        for sample in samples:
            symbols = sample["symbols"]
            obs_len = len(sample["ipa_chars"])
            if not symbols:
                continue
            # Выполняем forward и backward проходы
            alpha = self._forward_pass(symbols, obs_len)
            beta = self._backward_pass(symbols, obs_len)
            logZ = alpha[len(symbols), obs_len]  # Нормализующая константа
            if not math.isfinite(logZ):
                continue
            total_loglike += logZ
            # Вычисляем апостериорные вероятности для каждого символа и позиции
            for t, char in enumerate(symbols):
                log_len = self._get_log_length_probs(char)
                for pos in range(obs_len + 1):
                    if not math.isfinite(alpha[t, pos]):
                        continue
                    max_d = min(self.max_len, obs_len - pos)
                    for d in range(1, max_d + 1):
                        next_pos = pos + d
                        log_prob = log_len[d - 1]
                        if not math.isfinite(log_prob) or not math.isfinite(
                            beta[t + 1, next_pos]
                        ):
                            continue
                        contrib = alpha[t, pos] + log_prob + beta[t + 1, next_pos] - logZ
                        aposterior = math.exp(contrib)
                        if aposterior > 0:
                            length_counts[char][d - 1] += aposterior
        return length_counts, total_loglike

    # M-шаг алгоритма EM - обновление параметров модели (вероятности длин для каждого символа) на основе ожидаемых частот
    def maximization_step(self, length_counts):
        for ch in self.symbol_vocab:
            counts = length_counts[ch] + self.length_prior 
            probs = counts / counts.sum()
            self.length_log_probs[ch] = np.log(probs)

    # Обучение модели алгоритмом EM, выполняем итерации E-шага и M-шага до сходимости
    def fit(self, samples, n_iter=10, verbose=True):
        if not self.initialized:
            self.initialize_params(samples)
        history = []
        for iteration in range(1, n_iter + 1):
            length_counts, total_loglike = self.expectation_step(samples)
            self.maximization_step(length_counts)
            avg_ll = total_loglike / max(len(samples), 1)
            history.append(avg_ll)
            if verbose:
                print(f"Итерация {iteration}: средний log-likelihood = {avg_ll:.4f}")
        return history

    # Декодирование с помощью алгоритма Витерби (находит наиболее вероятную последовательность)
    def decode(self, sample):
        # Подготовка данных
        symbols = sample["symbols"]
        obs_len = len(sample["ipa_chars"])
        T = len(symbols)
        if T == 0:
            return []
        # Динамическое программирование для поиска оптимального пути
        dp = np.full((T + 1, obs_len + 1), -np.inf)
        backptr = [[None] * (obs_len + 1) for _ in range(T + 1)]
        dp[0, 0] = 0.0 
        # Прямым проходом ищем лучший путь
        for t in range(T): 
            char = symbols[t]
            log_len = self._get_log_length_probs(char) 
            for pos in range(obs_len + 1): 
                current = dp[t, pos]
                if not math.isfinite(current):
                    continue
                max_d = min(self.max_len, obs_len - pos)
                for d in range(1, max_d + 1):
                    log_prob = log_len[d - 1]
                    if not math.isfinite(log_prob):
                        continue
                    next_pos = pos + d
                    score = current + log_prob  
                    # Обновляем лучший путь
                    if score > dp[t + 1, next_pos]:
                        dp[t + 1, next_pos] = score
                        backptr[t + 1][next_pos] = (pos, d)
        if not math.isfinite(dp[T, obs_len]): # Проверяем, что найден путь от начала до конца
            return None
        # Восстанавливаем путь обратным проходом
        lengths = [0] * T
        pos = obs_len
        for t in range(T, 0, -1):
            prev = backptr[t][pos]
            if prev is None:
                return None
            prev_pos, d = prev
            lengths[t - 1] = d
            pos = prev_pos
        return lengths # Возвращаем список длин сегментов

In [ ]:
# Создание модели HSMM
hsmm = LengthHSMM(symbol_vocab, max_len=max_chunk_len, length_prior=0.2) # length_prior - параметр сглаживания для предотвращения нулевых вероятностей

# Обучение модели на 50 итерациях
training_history = hsmm.fit(samples, n_iter=50, verbose=True)

Итерация 1: средний log-likelihood = -1.6894
Итерация 2: средний log-likelihood = -1.5082
Итерация 3: средний log-likelihood = -1.4240
Итерация 4: средний log-likelihood = -1.3494
Итерация 5: средний log-likelihood = -1.2712
Итерация 6: средний log-likelihood = -1.1878
Итерация 7: средний log-likelihood = -1.1041
Итерация 8: средний log-likelihood = -1.0277
Итерация 9: средний log-likelihood = -0.9644
Итерация 10: средний log-likelihood = -0.9158
Итерация 11: средний log-likelihood = -0.8802
Итерация 12: средний log-likelihood = -0.8539
Итерация 13: средний log-likelihood = -0.8334
Итерация 14: средний log-likelihood = -0.8166
Итерация 15: средний log-likelihood = -0.8027
Итерация 16: средний log-likelihood = -0.7910
Итерация 17: средний log-likelihood = -0.7812
Итерация 18: средний log-likelihood = -0.7731
Итерация 19: средний log-likelihood = -0.7665
Итерация 20: средний log-likelihood = -0.7610
Итерация 21: средний log-likelihood = -0.7565
Итерация 22: средний log-likelihood = -0.75

In [23]:
# Функции для декодирования и построения предсказаний

# Функция для равномерного распределения длин сегментов между символами, используется как запасной вариант
def uniform_lengths(total_len: int, num_symbols: int, max_len: int):
    # Проверка входных данных
    if num_symbols == 0:
        return []
    if max_len <= 0:
        raise ValueError("max_len должен быть положительным")
    # Для каждого символа вычисляем минимальную и максимальную допустимую длину
    lengths = []
    remaining = total_len
    for idx in range(num_symbols):
        symbols_left = num_symbols - idx - 1
        min_len = max(0, remaining - symbols_left * max_len)
        max_len_allowed = min(max_len, remaining)
        if max_len_allowed < min_len:
            max_len_allowed = min_len
        # Вычисляем среднюю длину для оставшихся символов
        denom = symbols_left + 1
        avg = remaining / denom if denom > 0 else remaining
        length = int(round(avg))
        length = max(min_len, min(max_len_allowed, length))
        lengths.append(length)
        remaining -= length
    return lengths

# Функция для построения строки split_ipa из длин сегментов
# Объединяет сегменты фонем, принадлежащие одному слову, разделяя слова маркером <space>
def build_split_from_lengths(sample, lengths):
    ipa_chars = sample["ipa_chars"]
    word_ids = sample["word_ids"]
    pos = 0
    chunks_per_word = defaultdict(list)
    # Группируем сегменты по словам
    for length, word_idx in zip(lengths, word_ids): 
        chunk = "".join(ipa_chars[pos : pos + length])
        pos += length 
        chunks_per_word[word_idx].append(chunk) 
    # Объединяем сегменты внутри каждого слова
    words = []
    for w_idx in range(len(sample["words"])): 
        words.append("".join(chunks_per_word.get(w_idx, []))) 
    return "<space>".join(words) 


In [40]:
# Декодирование всех предложений с помощью обученной модели
pred_rows = []
failed_decodes = 0
for sample in samples:
    lengths = hsmm.decode(sample)
    # Проверяем корректность декодирования
    if lengths is None or sum(lengths) != len(sample["ipa_chars"]):
        failed_decodes += 1
        # Используем равномерное распределение как запасной вариант
        lengths = uniform_lengths(len(sample["ipa_chars"]), len(sample["symbols"]), hsmm.max_len)
    split_pred = build_split_from_lengths(sample, lengths)
    pred_rows.append(
        {
            "id": sample["id"],
            "split_phrase": sample["split_phrase"],
            "split_ipa_pred": split_pred,
            "split_ipa_true": sample["split_ipa_true"],
        }
    )

pred_df = pd.DataFrame(pred_rows)

In [ ]:
# Функция для подсчета Boundary F1-score - точность определения границ между словами
def ipa_boundaries(split_str: str):
    parts = split_str.split("<space>")
    offsets = []
    acc = 0
    for part in parts[:-1]:  # Последний сегмент не имеет границы после себя
        acc += len(part)
        offsets.append(acc)
    return set(offsets)

# Вычисляем F1
tp = 0
pred_total = 0
true_total = 0

for row in pred_df.itertuples():
    true_bounds = ipa_boundaries(row.split_ipa_true)
    pred_bounds = ipa_boundaries(row.split_ipa_pred)
    tp += len(true_bounds & pred_bounds)
    pred_total += len(pred_bounds)
    true_total += len(true_bounds)

boundary_f1 = (2 * tp / (pred_total + true_total)) if (pred_total + true_total) > 0 else 0.0

print(f"Boundary F1-score: {boundary_f1:.4f}")

Boundary F1-score: 0.8763


In [46]:
pred_df.head(-5)

,id,split_phrase,split_ipa_pred,split_ipa_true
0,0,取り残さ れ,toɾʲinokosa<space>ɾe,toɾʲinokosa<space>ɾe
1,1,ブス か わ,bɯsɨ<space>ka<space>wa,bɯsɨ<space>ka<space>wa
2,2,早鐘 を 打つ,hajaɡane<space>o<space>ːtsɨ,hajaɡane<space>o<space>ːtsɨ
3,3,マニラ 紙,maɲiɾa<space>ɕi,maɲiɾa<space>ɕi
4,4,すいこめ ば,sɨikome<space>ba,sɨikome<space>ba
...,...,...,...,...
5864,5864,プレイ 中,pɯɾeː<space>tɕɨː,pɯɾeː<space>tɕɨː
5865,5865,ほこっ た,hokot<space>ːa,hokotː<space>a
5866,5866,いかに し て,ikaɲi<space>ɕi<space>te,ikaɲi<space>ɕi<space>te
5867,5867,草木 も なびく,kɯsaci<space>mo<space>nabʲikɯ,kɯsaci<space>mo<space>nabʲikɯ
